In [1]:
junction_id = {'KLONGTEI': 'cluster_1892287670_272491964_272492178',
      'RAMA4': 'cluster_272488163_282390730_66263210_66263222',
      'NARANONG': 'cluster_272488164_272492179_3457051443_61907354',
      'SUNLAKAKHON': 'gneJ83',
      'KASEMRAT': 'cluster_272448137_272555800_272555808_7660045934_7710268409',
      'ATTHAKAWI_RAMA4' : '270329335'}
junction_name = list(junction_id.keys())

In [2]:
# import gym
# from gym import error, spaces
import gym
import lxml.etree as ET
import csv
import json
import os, sys
os.environ['SUMO_HOME']='/usr/share/sumo'
tools = os.path.join(os.environ['SUMO_HOME'],'tools')
sys.path.append(tools)
import traci
import traci.constants as tc
import numpy as np
from sumolib import checkBinary
import datetime
import matplotlib
import matplotlib.pyplot as plt
import torch
import copy
import random
import pandas as pd
from multiprocessing.pool import ThreadPool
from itertools import zip_longest

In [3]:
namelane_csv = pd.read_csv('namelane_KASEMRAT_nodot.csv')
namelane_df = pd.DataFrame(namelane_csv, columns = ['name' , 'id']).dropna()
NAME = namelane_df.set_index('name')
ID = namelane_df.set_index('id')
if NAME.loc['KASEMRAT_EB_0_0_XSXX','id'] == '459551209#3_0':
    print('ok')

ok


In [4]:
NAME.loc[:,'id']

name
KASEMRAT_EB_0_0_XSXX                      459551209#3_0
KASEMRAT_EB_0_1_XSXX                      459551209#3_1
KASEMRAT_EB_0_2_XSXX                      459551209#3_2
KASEMRAT_EB_0_3_XSRT                      459551209#3_3
KASEMRAT_EB_1_0_LSXX                      459551209#0_0
                                              ...      
MASUKGRIDLOCK_ATTHAKAWI_SB_2_1_LXXX        27702347#6_1
MASUKGRIDLOCK_ATTHAKAWI_SB_1_0_LSXT        27702347#4_0
MASUKGRIDLOCK_ATTHAKAWI_SB_2_0_LXXX        27702347#6_0
MASUKGRIDLOCK_SUKHUMVUT22_SB_0_0_XSXX    -453669106#1_0
MASUKGRIDLOCK_SUKHUMVUT24_SB_0_0_XSXX     328942767#2_0
Name: id, Length: 317, dtype: object

In [5]:
ID.loc['820373198#0_3', 'name']

'RAMA4_WB_2_3_XSXX'

In [6]:
namedetector_csv = pd.read_csv('namedetector_KASEMRAT_flow.csv')
namedetector_df = pd.DataFrame(namedetector_csv, columns = ['name' , 'id'])
NAME_D = namedetector_df.set_index('name')
ID_D = namedetector_df.set_index('id')
listdetector = open("namedetector_KASEMRAT_flow.txt", "r")
detector = {}
for l in listdetector:
    l = l.strip().split(' ')
    if len(l)> 1:
        d = []
        for detec in l[2:]:
            if type(NAME_D.loc[detec,'id']) == str:
                d.append(NAME_D.loc[detec,'id'])
            else : d.append(NAME_D.loc[detec,'id'][0])
    if str(l[0])!= '':
        detector[str(l[0])] = d
list_detector= list(detector.keys())

In [7]:
NAME_D.head()

,id
name,
RAMA4_EB_1_4_XSXX,D459492917#0_4
KLONGTEI_EB_0_0_XSXX,D825786400_0
KLONGTEI_EB_0_1_XSXX,D825786400_1
KLONGTEI_EB_0_2_XSXX,D825786400_2
KLONGTEI_EB_0_3_XSXX,D825786400_3


In [8]:
# type(NAME_D.loc['KLONGTEI_EB_0_0_XSXX','id'])
# for i in range(len(list(detector.keys()))):
#     print(i , list(detector.keys())[i])

In [9]:
def get_hot_encoding_current_phase():
    number_phase = [4,9,7,5,4,3]
    current_phase = [traci.trafficlight.getPhase(junction_id[key]) for key in junction_id.keys()]
#     current_phase = [0,2,1,1,1,1]
    hot_encoding_current_phase = np.array([])
    for i in range(len(current_phase)):
        binary_phase = np.zeros(number_phase[i])
        binary_phase[current_phase[i]] = 1
        hot_encoding_current_phase = np.concatenate((hot_encoding_current_phase, binary_phase), axis=None)
    return current_phase, hot_encoding_current_phase


In [10]:
# ID.loc['']#, '820373198#0', '820373196#0', '482209831#0']

In [11]:
# MAP_JUNCTIONNAME = dict 
# 	key = current phase
# 	value = list compose of
# 		1. name of detector at a particular critical spot that traffic police pay attention to
# 			(detector at that phase (check to end phase) + detector at priority phase(change to priority phase))
# 		2. threshold or -threshold of that spot:
# 			 occupancy = get_occupancy_average_percent(detector_name)
# 			threshold#1 -threshold
# 				if -threshold  <= 0:
#                 					if occupancy >= (-1)* -threshold :
#                     					Index_detector = list_detector.index(detector_name) 
#                     					state_attention[Index_detector] = 1
#           threshold#2 threshold
# 				elif occupancy <= threshold :
#                 					Index_detector = list_detector.index(detector_name) 
#                 					state_attention[Index_detector] = 1

In [12]:
def get_state_attention(current_phase, hot_encoding_current_phase):
#     current_phase = [KLONGTEI phase, RAMA4 phase,NARANONG phase, SUNLAKAKHON phase, KASEMRAT phase, ATTHAKAWI_RAMA4 phase]
    MAP_RAMA4 = {0: [['RAMA4_EB_FPX_TP1',-50] , ['KASEMRAT_EB_FPX_TP2_RAMA4',-90], ['NARANONG_SW_FPX_TP1',-90]],
                 1: [['RAMA4_EB_FP1_TP3',10], ['RAMA4_NB_FPX_TP5',-83]],
                 2: [['RAMA4_WB_FP2_TP4',10], ['RAMA4_EB_FPX_TP1',-50], ['RAMA4_NB_FPX_TP5',-83]],
                3: [['RAMA4_EB_FP3_TP2',40], ['RAMA4_WB_FP3_TP2',-80], ['RAMA4_EB_FPX_TP1',-50],
                    ['RAMA4_NB_FPX_TP5',-83]],
                4: [['RAMA4_SB_FP4_TP5',10], ['RAMA4_EB_FPX_TP1',-50], ['RAMA4_NB_FPX_TP5',-83 ]],
                5: [['RAMA4_NB_FP5_TP1',5], ['RAMA4_EB_FPX_TP1',-50]],
                6: [['KASEMRAT_EB_FPX_TP2_RAMA4',-90]],
                7: [['KASEMRAT_EB_FPX_TP2_RAMA4',-90]],
                8: [['KASEMRAT_EB_FPX_TP2_RAMA4',-90]]}
    MAP_KLONGTEI = {0: [['NARANONG_SB_FP5_TP6',-20], ['RAMA4_EB_FP1_TP3',10]],
                    1: [['RAMA4_EB_FP3_TP2',40], ['RAMA4_EB_FP1_TP3',10]],
                    2: [['KLONGTEI_NB_FP3_TPX',10]],
                   3: [['RAMA4_EB_FP3_TP2',40], ['RAMA4_EB_FP1_TP3',10]] }
    MAP_NARANONG = {0: [['RAMA4_NB_FPX_TP5',-83]],
                1: [['NARANONG_SB_FP1_TP2',30], ['NARANONG_EB_FP1_TP2',-50], ['NARANONG_WB_FPX_TP2',-30]],
                2: [['NARANONG_WB_FP2_TP3',5], ['NARANONG_WB_FP2_TP3_FLOW',10], ['NARANONG_SW_FPX_TP1',-30],
                   ['NARANONG_WB_FPX_TP2',-30]],
                3: [['NARANONG_EB_FP3_TP4',10], ['NARANONG_SW_FPX_TP1',-30], ['NARANONG_WB_FPX_TP2',-30]],
                4: [['NARANONG_WB_FP4_TP5',5], ['NARANONG_SW_FPX_TP1',-30], ['NARANONG_WB_FPX_TP2',-30]],
                5: [['NARANONG_SB_FP5_TP6',5], ['NARANONG_SW_FPX_TP1',-30],['NARANONG_WB_FPX_TP2',-30]],
                6: [['NARANONG_NB_FP6_TP1',5], ['NARANONG_SW_FPX_TP1',-30], ['NARANONG_WB_FPX_TP2',-30]] }
    MAP_SUNLAKAKHON = {0: [['NARANONG_WB_FPX_TP2',-30]],
                    1: [['SUNLAKAKHON_SB_FP1_TPX',10]],
                    2: [['SUNLAKAKHON_NB_FP2_TP3',10], ['SUNLAKAKHON_SB_FPX_TP1',-27.5]],
                    3:[['SUNLAKAKHON_EB_FP3_TP4',-45], ['SUNLAKAKHON_WB_FP3_TP4',10], ['SUNLAKAKHON_SB_FPX_TP1',-27.5]],
                    4:[['SUNLAKAKHON_SB_FP4_TP1',10], ['SUNLAKAKHON_SB_FPX_TP1',-27.5]] }
    MAP_KASEMRAT = {0: [['SUNLAKAKHON_SB_FPX_TP1',-20], ['MASUKGRIDLOCK_ARI_NB_FPX_TP2',20]],
                    1:[['KASEMRAT_EB_FPX_TP2',-65], ['KASEMRAT_NB_FPX_TP3',-30], ['MASUKGRIDLOCK_ARI_NB_FPX_TP2',-50],\
                       ['MASUKGRIDLOCK_SUKHUMVUT_FPX_TP2',-50]],
                    2:[['KASEMRAT_EB_FP2_TP1',20], ['KASEMRAT_NB_FPX_TP3',-30]],
                    3:[['KASEMRAT_EB_FPX_TP2',-65], ['MASUKGRIDLOCK_ARI_NB_FPX_TP2',-50], ['MASUKGRIDLOCK_SUKHUMVUT_FPX_TP2',-50]]}
    MAP_ATTHAKAWI_RAMA4 = {0:[['KASEMRAT_EB_FPX_TP2',-50]],
                           1: [['MASUKGRIDLOCK_SUKHUMVUT_FPX_TP2',-50], ['MASUKGRIDLOCK_ARI_NB_FPX_TP2', -50], ['KASEMRAT_EB_FPX_TP2_RAMA4',-50]],
                          2:[['KASEMRAT_EB_FPX_TP2_RAMA4',-50]]}
    MAP = [MAP_KLONGTEI, MAP_RAMA4, MAP_NARANONG, MAP_SUNLAKAKHON,  MAP_KASEMRAT, MAP_ATTHAKAWI_RAMA4]
    state_attention = np.zeros(31)
    for i in range(len(current_phase)):
        for e in MAP[i][current_phase[i]]:
            occupancy = get_occupancy_average_percent(detector[e[0]]) 
            if e[1] <= 0:
                if occupancy >= (-1)*e[1]:
                    Index_detector = list_detector.index(e[0]) 
                    state_attention[Index_detector] = 1
            elif occupancy <= e[1]:
                Index_detector = list_detector.index(e[0]) 
                state_attention[Index_detector] = 1
#     print(state_attention)
    state = np.concatenate((state_attention, hot_encoding_current_phase), axis=None)
    return state


In [13]:
def read_summary_xml(dateTimeObj):
    meanWaitingTime = []
    meanTravelTime = []
    meanSpeed = []
    tree = ET.parse('summary/summary3'+dateTimeObj+'.xml')
    summary = tree.getroot()
    for step in summary:
        list1 = step.attrib
        meanWaitingTime.append(float(list1["meanWaitingTime"]))
        meanTravelTime.append(float(list1["meanTravelTime"]))
        meanSpeed.append(float(list1["meanSpeed"]))
    meanWaitingTime_avg = sum(meanWaitingTime)/len(meanWaitingTime)
    meanTravelTime_avg = sum(meanTravelTime)/len(meanTravelTime)
    meanSpeed_avg = sum(meanSpeed)/len(meanSpeed)
    os.remove('summary/summary3'+dateTimeObj+'.xml')
    return meanWaitingTime_avg,meanTravelTime_avg,meanSpeed_avg

In [14]:
#reset the environment
def start(dateTimeObj):
    sumoBinary = checkBinary('sumo')
    traci.start([sumoBinary, "-c", "KASEMRAT-SUMO-UsingBookNetFile/osm.sumocfg",
#                              "--summary-output"
#                  , "summary/summary3"+dateTimeObj+".xml"
#                  ,
                 '--start','true','--quit-on-end','true','--time-to-teleport','-1',
                '--lanechange.duration', '0.1'])

In [15]:
def get_occupancy_average_percent(detector_id): 
    #get occupancy average for all detector in list of detector_id and scale by (Vehicle Length + MinimumGap)/MinimumGap 
    #Vehicle Length = 4.62 MinimumGap = 2.37
    occupancy = (sum([traci.lanearea.getLastStepOccupancy(e)*traci.lanearea.getLength(e) for e in detector_id]))/\
                 sum([traci.lanearea.getLength(e) for e in detector_id])*((4.62+2.37)/4.62)
#     print(occupancy)
    return occupancy

In [16]:
def get_throughput():
    loopID = traci.inductionloop.getIDList()
    throughput = sum([traci.inductionloop.getLastStepVehicleNumber(i) for i in loopID if traci.inductionloop.getLastStepMeanSpeed(i) > 0])  #if traci.inductionloop.getLastStepMeanSpeed(i) > 0
    return throughput

In [17]:
def get_drawback():
#     laneID = traci.lane.getIDList()
    drawback = sum([traci.lanearea.getLastStepVehicleNumber(i) for i in NAME.loc[:,'id']])
    return drawback

In [18]:
def get_reward():
    throughput = 0
    for i in range(5):
        traci.simulationStep()
        throughput += get_throughput()
    
    drawback = get_drawback()
    reward = throughput - 0.004*drawback
    return reward, throughput, drawback

In [19]:
number_phase = [4,9,7,5,4,3]
def set_current_phase(action, current_phase):
    if action < 4:
        phase = action
        current_phase[0] = phase
    elif action < 13:
        phase = (action-4)
        current_phase[1] = phase
    elif action < 20:
        phase = (action-13)
        current_phase[2] = phase
    elif action < 25:
        phase = (action-20)
        current_phase[3] = phase
    elif action < 29:
        phase = (action-25)
        current_phase[4] = phase
    else:
        phase = (action-29)
        current_phase[5] = phase
    for i in range (6):
        traci.trafficlight.setPhase(junction_id[junction_name[i]], current_phase[i])
    
    if current_phase[5] == 2:
        traci.lane.setAllowed(NAME.loc['MASUKGRIDLOCK_ATTHAKAWI_SB_1_0_LSXT', 'id'], ['passenger'])
        traci.lane.setAllowed(NAME.loc['MASUKGRIDLOCK_ATTHAKAWI_SB_2_0_LXXX', 'id'], ['passenger'])
        traci.lane.setAllowed(NAME.loc['MASUKGRIDLOCK_ATTHAKAWI_SB_0_0_XSRT', 'id'], ['passenger'])
    elif current_phase[5] == 1:
        traci.lane.setDisallowed(NAME.loc['MASUKGRIDLOCK_ATTHAKAWI_SB_1_0_LSXT', 'id'], ['passenger'])         
        traci.lane.setDisallowed(NAME.loc['MASUKGRIDLOCK_ATTHAKAWI_SB_2_0_LXXX', 'id'], ['passenger'])
        traci.lane.setDisallowed(NAME.loc['MASUKGRIDLOCK_ATTHAKAWI_SB_0_0_XSRT', 'id'], ['passenger'])
        
    return 

In [20]:
''.join(str(datetime.datetime.now()).split(' '))

'2021-04-2414:09:31.371135'

In [21]:
reward_memory = []
def plot_durations():
    print('show')
    plt.figure(2)
    plt.clf()
    durations_t = torch.tensor(reward_memory, dtype=torch.float)
    plt.title('Training...')
    plt.xlabel('Step')
    plt.ylabel('Reward')
    plt.plot(durations_t.numpy())
    # Take 100 episode averages and plot them too
    if len(durations_t) >= 100:
        means = durations_t.unfold(0, 100, 1).mean(1).view(-1)
        means = torch.cat((torch.zeros(99), means))
        plt.plot(means.numpy())

    plt.pause(0.001)  # pause a bit so that plots are updated

In [22]:
class MyEnv3(gym.Env):
    def __init__(self, env_config):
        self.episode = 0
        self.count = 0
        self.reward = 0
        self.rewards = 0
        self.throughputs = 0
        self.drawbacks = 0
        self.current_phase = [1,1,1,1,1,1]
        self.done = False
        self.reward_memory = []
        self.log_action = []
        self.action_space = gym.spaces.Discrete(32)
        self.observation_space = gym.spaces.Box(low=0, high=1, shape=(63,), dtype=np.int8)
        self.dateTimeObj = ''
        print(self.dateTimeObj)
        with open( "./Raytest/ray_results/resultstatethreshold2.csv" , 'w', newline='') as csv_file:
                header = ['rewards', 'throughput','backlog',"meanWaitingTime", "meanTravelTime","meanSpeed","action"]
                writer = csv.DictWriter(csv_file, fieldnames = header)
                writer.writeheader()
        
    def reset(self):
        
        self.episode += 1
        self.log_action = []
        self.reward_memory.append(self.rewards)
        self.count = 0
        dateTimeObj = datetime.datetime.now()
        self.dateTimeObj = dateTimeObj.strftime("%d-%b-%Y-%H-%M-%S-%f")
#         print(type(self.dateTimeObj))
        start(self.dateTimeObj)
        self.reward = 0
        self.rewards = 0
        self.throughputs = 0
        self.backlogs = 0
        print(self.reward_memory)
        current_phase, hot_encoding_current_phase = get_hot_encoding_current_phase()
        state = get_state_attention(current_phase, hot_encoding_current_phase)
#         print(state)
        self.done = False
        self.current_phase = current_phase
        if len(self.reward_memory)%100 == 0:
            print('memory',self.reward_memory[-10:])
        return state 
    
    
    def step(self, action):
        set_current_phase(action, self.current_phase)
#         print(action)
        current_phase, hot_encoding_current_phase = get_hot_encoding_current_phase()
        self.reward, throughput, drawback= get_reward()
        self.rewards += self.reward
        self.throughputs += throughput
        self.drawbacks += drawback
        state = get_state_attention(current_phase, hot_encoding_current_phase)
        if np.isnan(self.reward) == True:
            print('HELP', type(self.reward))
        self.count += 1
        self.current_phase = current_phase
        self.log_action.append(str(action))
#         print('count', self.count)
        self.done = False
        if self.count >= 2880: #2880
            traci.close()
            self.done = True
#             meanWaitingTime_avg,meanTravelTime_avg,meanSpeed_avg = read_summary_xml(self.dateTimeObj)
            meanWaitingTime_avg = 0
            meanTravelTime_avg = 0
            meanSpeed_avg = 0
            data_set = {"rewards": self.rewards, "meanWaitingTime": meanWaitingTime_avg, "meanTravelTime": meanTravelTime_avg,
                       "meanSpeed": meanSpeed_avg, "throughput": self.throughputs, "backlog": self.backlogs/2880, 
                        "action":self.log_action
                       }
            
            self.json_dump = json.dumps(data_set)
#             if self.episode%20 == 0:
#                 self.log_action_json = json.dumps(self.log_action)
#                 with open( "./Raytest/ray_results/action"+self.dateTimeObj+".csv" , 'a', newline='') as csv_file:
#                     header = ['action']
#                     writer = csv.DictWriter(csv_file, fieldnames = header)
#                     writer.writerow({'action': self.log_action})
                                 
            with open( "./Raytest/ray_results/resultstatethreshold2.csv" , 'a', newline='') as csv_file:
                header = ['rewards', 'throughput','backlog',"meanWaitingTime", "meanTravelTime","meanSpeed","action"]
                writer = csv.DictWriter(csv_file, fieldnames = header)
                writer.writerow({'rewards': self.rewards, 
                                 'throughput': self.throughputs,
                                'backlog': self.backlogs/2880,
                                "meanWaitingTime": meanWaitingTime_avg, "meanTravelTime": meanTravelTime_avg,
                       "meanSpeed": meanSpeed_avg, "action":self.log_action})

        
        return_state = np.array(state).astype(np.float16)
#         print(return_state)
        info = {"throughput": throughput,
                "drawback":drawback
                }
        info = {**info}
#         print(info)
        return np.int8(return_state) , self.reward, self.done, info

In [23]:
# if __name__ == "__main__":
#     print("asd")
#     env = MyEnv3(env_config=None)

#     for i_episode in range(1):
#         observation = env.reset()
#         for t in range(1000):
#             # env.render()
# #             print(observation)
#             action = env.action_space.sample()
# #             print('action' ,action)
#             observation, reward, done, info = env.step(action)
#             if done:
#                 print("Episode finished after {} timesteps")
#                 break
#     env.close()


In [24]:
# traci.close()

In [ ]:
import ray
import ray.tune  as tune
from ray.tune import Callback
from ray.rllib.utils import merge_dicts
from ray.rllib.agents.dqn.dqn import calculate_rr_weights, \
    DEFAULT_CONFIG as DQN_CONFIG, DQNTrainer, validate_config
# env = gym.make('CartPole-v0')


class MyCallback(Callback):
    def on_trial_start(self, iteration, trials, trial, **info):
        print(f"I am in callback. This is iteration {iteration} inside trial {trial}")
#         dateTimeObj = datetime.datetime.now()
#         dateTimeObj = dateTimeObj.strftime("%d-%b-%Y-%H-%M-%S-%f")
#         with open( "./Raytest/ray_results/"+dateTimeObj+".csv" , 'a', newline='') as csv_file:
#                 header = ['rewards', 'throughput','drawback']
#                 writer = csv.DictWriter(csv_file, fieldnames = header)
#                 writer.writeheader()
#         print(info)
    def on_trial_result(self, iteration, trials, trial, result, **info):
        print("I am in second callback. Got result:", info)
#         with open( "./Raytest/ray_results/"+str(trial)+".csv" , 'a', newline='') as csv_file:
#             header = ['rewards']
#             writer = csv.DictWriter(csv_file, fieldnames = header)
#             writer.writeheader()
#             writer.writerow({'rewards': result["episode_reward_mean"]})
            

def trial_name_id(trial):
    return f"{trial.trainable_name}_{trial.trial_id}"

if __name__ == "__main__":
    ray.shutdown()
    ray.init(
#              object_store_memory=int(1e9),  # 4gb
#              redis_max_memory=int(1e9)  #2gb
             )
    experiment_spec = tune.Experiment(
        trial_name_creator=trial_name_id,
        name = "experiment_apex6",
        run = "APEX",
        local_dir = "./Raytest/ray_results",
        checkpoint_freq = 3,
        checkpoint_at_end = True,
        log_to_file=True,
        config = {
#         "optimizer": merge_dicts(
#             DQN_CONFIG["optimizer"], {
#                 "max_weight_sync_delay": 400,
#                 "num_replay_buffer_shards": 4,
#                 "debug": False
#             }),
#         "n_step": 3,
        
            "adam_epsilon": 1e-8,
            # If not None, clip gradients during optimization at this value
            
        "num_gpus": 0,
        "dueling": True,
        "double_q": True,
        "num_workers": 31,
        "buffer_size": 100000,
        "framework": "tf",
        "learning_starts": 28800, #2160
        "train_batch_size": 2880,
#             "num_samples": 20,
        "rollout_fragment_length": 50,
        "target_network_update_freq": 28800,
        "prioritized_replay": True,
        "timesteps_per_iteration":11520, #2880
#         "exploration_config": {"type": "PerWorkerEpsilonGreedy"},
#         "worker_side_prioritization": True,
#         "min_iter_time_s": 30,
        # If set, this will fix the ratio of replayed from a buffer and learned
        # on timesteps to sampled from an environment and stored in the replay
        # buffer timesteps. Otherwise, replay will proceed as fast as possible.
#         "training_intensity": None,
         "worker_side_prioritization": True,
         "lr": 5e-4,
        "gamma": 0.9,

            
            "env": MyEnv3 ,

            
            }
        
        )
    results = tune.run_experiments(experiment_spec, 
                                  callbacks=[MyCallback()])


Instructions for updating:
non-resource variables are not supported in the long term


2021-04-24 14:09:33,492	INFO services.py:1172 -- View the Ray dashboard at http://127.0.0.1:8266
2021-04-24 14:09:33,495	WARNING services.py:1619 -- WARNING: The object store is using /tmp instead of /dev/shm because /dev/shm has only 67096576 bytes available. This will harm performance! You may be able to free up space by deleting files in /dev/shm. If you are inside a Docker container, you can increase /dev/shm size by passing '--shm-size=Xgb' to 'docker run' (or add it to the run_options list in a Ray cluster config). Make sure to set this to more than 2gb.
2021-04-24 14:09:35,224	ERROR syncer.py:72 -- Log sync requires rsync to be installed.


I am in callback. This is iteration 0 inside trial APEX_b2b33_00000


Trial name,status,loc
APEX_b2b33_00000,RUNNING,


(pid=31764) WARNING:tensorflow:From /usr/local/lib/python3.8/dist-packages/tensorflow/python/compat/v2_compat.py:96: disable_resource_variables (from tensorflow.python.ops.variable_scope) is deprecated and will be removed in a future version.
(pid=31764) Instructions for updating:
(pid=31764) non-resource variables are not supported in the long term
(pid=31764) 2021-04-24 14:09:37,099	INFO trainer.py:616 -- Tip: set framework=tfe or the --eager flag to enable TensorFlow eager execution
(pid=31764) 2021-04-24 14:09:37,099	INFO trainer.py:641 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.
(pid=31759) WARNING:tensorflow:From /usr/local/lib/python3.8/dist-packages/tensorflow/python/compat/v2_compat.py:96: disable_resource_variables (from tensorflow.python.ops.variable_scope) is deprecated and will be removed in a future version.
(pid=31759) Instructions for updating:
(pid=31759) non-resource variables are not supported in 

...

Trial name,status,loc,iter,total time (s),ts,reward,episode_reward_max,episode_reward_min,episode_len_mean
APEX_b2b33_00000,RUNNING,10.42.2.141:31764,24,8239.78,293700,-21087.2,-13873.5,-25210.6,2880


I am in second callback. Got result: {}
Result for APEX_b2b33_00000:
  custom_metrics: {}
  date: 2021-04-24_16-32-31
  done: false
  episode_len_mean: 2880.0
  episode_reward_max: -13873.507999999969
  episode_reward_mean: -21087.211636363605
  episode_reward_min: -25210.576000000092
  episodes_this_iter: 0
  episodes_total: 33
  experiment_id: 6b40ebf539b04fc894c7093ce80dd64e
  hostname: raytest4-866cbb9cc-69x4h
  info:
    exploration_infos:
    - cur_epsilon: 0.0
    - cur_epsilon: 0.4000000059604645
    - cur_epsilon: 0.3230035603046417
    - cur_epsilon: 0.26082825660705566
    - cur_epsilon: 0.21062114834785461
    - cur_epsilon: 0.17007845640182495
    - cur_epsilon: 0.13733987510204315
    - cur_epsilon: 0.11090317368507385
    - cur_epsilon: 0.0895553007721901
    - cur_epsilon: 0.07231670618057251
    - cur_epsilon: 0.058396387845277786
    - cur_epsilon: 0.047155603766441345
    - cur_epsilon: 0.03807856887578964
    - cur_epsilon: 0.03074878640472889
    - cur_epsilon: 0.0

Trial name,status,loc,iter,total time (s),ts,reward,episode_reward_max,episode_reward_min,episode_len_mean
APEX_b2b33_00000,RUNNING,10.42.2.141:31764,25,8564.04,305250,-21087.2,-13873.5,-25210.6,2880


I am in second callback. Got result: {}
Result for APEX_b2b33_00000:
  custom_metrics: {}
  date: 2021-04-24_16-38-33
  done: false
  episode_len_mean: 2880.0
  episode_reward_max: -13873.507999999969
  episode_reward_mean: -21087.211636363605
  episode_reward_min: -25210.576000000092
  episodes_this_iter: 0
  episodes_total: 33
  experiment_id: 6b40ebf539b04fc894c7093ce80dd64e
  hostname: raytest4-866cbb9cc-69x4h
  info:
    exploration_infos:
    - cur_epsilon: 0.0
    - cur_epsilon: 0.4000000059604645
    - cur_epsilon: 0.3230035603046417
    - cur_epsilon: 0.26082825660705566
    - cur_epsilon: 0.21062114834785461
    - cur_epsilon: 0.17007845640182495
    - cur_epsilon: 0.13733987510204315
    - cur_epsilon: 0.11090317368507385
    - cur_epsilon: 0.0895553007721901
    - cur_epsilon: 0.07231670618057251
    - cur_epsilon: 0.058396387845277786
    - cur_epsilon: 0.047155603766441345
    - cur_epsilon: 0.03807856887578964
    - cur_epsilon: 0.03074878640472889
    - cur_epsilon: 0.0

Trial name,status,loc,iter,total time (s),ts,reward,episode_reward_max,episode_reward_min,episode_len_mean
APEX_b2b33_00000,RUNNING,10.42.2.141:31764,26,8926.61,316800,-21087.2,-13873.5,-25210.6,2880


In [ ]:
len([1., 0., 0., 0., 0., 0., 0., 1., 1., 0., 0., 1., 1., 0., 0., 0., 0.,
       1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 0., 0., 1.,
       0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
       1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 1.])

In [ ]:
x = np.array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
       0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       1., 0., 0., 1., 0., 0.])

In [ ]:
len(x)